# Compute Complexity metrics on all models

In [1]:
from TSInterpret.InterpretabilityModels.Saliency.TSR import TSR, Saliency_PTY
from TSInterpret.InterpretabilityModels.counterfactual.TSEvoCF import TSEvo
# from TSInterpret.InterpretabilityModels.counterfactual.SETSCF import SETSCF

import torch 
from XTSCBench.ClassificationModels.CNN_T import ResNetBaseline, UCRDataset,fit
from XTSCBench.ClassificationModels.LSTM import LSTM
from XTSCBench.CounterfactualEvaluation import CounterfactualEvaluation
from tslearn.datasets import UCR_UEA_datasets
import sklearn
import numpy as np 
import os


2024-12-02 01:54:22.733184: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-02 01:54:22.740262: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-02 01:54:22.780765: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-02 01:54:22.780795: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-02 01:54:22.780817: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to regi

## Data Loading

In [59]:
dataset='Epilepsy'
# dataset = 'ptbxl'

In [60]:



if dataset in ['ECG200','ECG5000','Epilepsy']:
    train_x, train_y, test_x, test_y=UCR_UEA_datasets().load_dataset(dataset)
elif dataset == 'ptbxl':
    train_x = np.load(f'./datasets/ptbxl/x_train.npy')
    train_y = np.load(f'./datasets/ptbxl/y_train.npy')
    test_x = np.load(f'./datasets/ptbxl/x_test.npy')
    test_y = np.load(f'./datasets/ptbxl/y_test.npy')

# 1 hot encoding outcomes
enc1=sklearn.preprocessing.OneHotEncoder(sparse=False).fit(np.vstack((train_y.reshape(-1,1),test_y.reshape(-1,1))))
train_y=enc1.transform(train_y.reshape(-1,1))
test_y=enc1.transform(test_y.reshape(-1,1))    

n_pred_classes =train_y.shape[1]
NumTimesteps = train_x.shape[-2]
NumFeatures = train_x.shape[-1]

#For use with CNN reverse Data Dimensions
# Note that (samples, timesteps, features) -> (samples, features, timesteps) for CNN models
train_dataset = UCRDataset(train_x.astype(np.float64).reshape(-1,NumFeatures,NumTimesteps),train_y.astype(np.int64))
test_dataset = UCRDataset(test_x.astype(np.float64).reshape(-1,NumFeatures,NumTimesteps),test_y.astype(np.int64))
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=16,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=1,shuffle=False)


In [61]:
train_dataset.feature.shape

(137, 3, 206)

In [62]:
train_x.shape, train_y.shape, test_x.shape, test_y.shape, n_pred_classes

((137, 206, 3), (137, 4), (138, 206, 3), (138, 4), 4)

In [63]:
train_dataset.feature.shape

(137, 3, 206)

## Load/Train Model

In [64]:
device='cpu'

model_path = './trained_models'

# Model saved by this name
model_name = f'cnn_{dataset}'
NumFeatures, NumTimesteps

(3, 206)

In [65]:


if os.path.isfile(f'./{model_path}/{model_name}'):
    model = ResNetBaseline(in_channels=NumFeatures, num_pred_classes=n_pred_classes)
    model = torch.load(f'./{model_path}/{model_name}')
    print(f"Model {model_name} successfully loaded")
else:
    print("Model not found. Please train model using training_models.ipynb and provide in this notebook")

model.eval()


Model cnn_Epilepsy successfully loaded


ResNetBaseline(
  (layers): Sequential(
    (0): ResNetBlock(
      (layers): Sequential(
        (0): ConvBlock(
          (layers): Sequential(
            (0): Conv1dSamePadding(3, 64, kernel_size=(8,), stride=(1,))
            (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU()
          )
        )
        (1): ConvBlock(
          (layers): Sequential(
            (0): Conv1dSamePadding(64, 64, kernel_size=(5,), stride=(1,))
            (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU()
          )
        )
        (2): ConvBlock(
          (layers): Sequential(
            (0): Conv1dSamePadding(64, 64, kernel_size=(3,), stride=(1,))
            (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU()
          )
        )
      )
      (residual): Sequential(
        (0): Conv1dSamePadding(3, 64, kernel_size=(1,)

# Make explainers

In [66]:
from TSInterpret.InterpretabilityModels.counterfactual.TSEvoCF import TSEvo

tsevo_exp = TSEvo(model= model,data=(train_x,train_y), mode = 'feat',backend='PYT',epochs=30)

y was one Hot Encoded


In [67]:
NumTimesteps, NumFeatures

(206, 3)

In [68]:
# Temporal Saliency 

## Methods
# * Gradients (GRAD)
# * Integrated Gradients (IG)
# * Gradient Shap (GS)
# * DeepLift (DL)
# * DeepLiftShap (DLS)
# * SmoothGrad (SG)
# * Shapley Value Sampling(SVS)
# * Feature Ablation (FA)
# * Occlusion (FO)

from TSInterpret.InterpretabilityModels.Saliency.TSR import TSR, Saliency_PTY

tsr_GRAD_exp = Saliency_PTY(model, NumTimeSteps=NumTimesteps, NumFeatures=NumFeatures, method='GRAD', mode='feat', tsr=True)


In [69]:
tsr_FA_exp = Saliency_PTY(model, NumTimeSteps=NumTimesteps, NumFeatures=NumFeatures, method='FA', mode='feat', tsr=True)


In [70]:
tsr_FO_exp =  Saliency_PTY(model, NumTimeSteps=NumTimesteps, NumFeatures=NumFeatures, method='FO', mode='feat', tsr=True)

In [71]:
# # NativeGuideCF
# from TSInterpret.InterpretabilityModels.counterfactual.NativeGuideCF import NativeGuideCF

# ng_exp = NativeGuideCF(model,(train_x,train_y), backend='PYT', mode='feat',method='NUN_CF')

In [73]:
explainer =  [
    tsevo_exp,
    tsr_FA_exp,
    tsr_GRAD_exp,
    # tsr_FO_exp
    ]


In [15]:

bm=CounterfactualEvaluation(explainer=explainer)


In [16]:
SummaryTable = bm.evaluate(test_x[0:2], np.argmax(test_y[0:2],axis=1),model, mode='feat',aggregate=True)


In [17]:
SummaryTable.head()

,d1_mean,d2_mean,d3_mean,d4_mean,validty_mean,d1_std,d2_std,d3_std,d4_std,validty_std,method,normalize,tsr
0,1.0,0.749141,164.691622,4.681957,1.0,0.0,0.073618,32.029047,0.210359,0.0,GRAD,True,True


# Metric Settings

In [74]:
test_y.shape

(138, 4)

In [75]:
num_test_samples = 2
interp_folder = './interp_metrics'

# Counterfactual Metrics

In [48]:
bm=CounterfactualEvaluation(explainer=explainer)


In [49]:
SummaryTable_counterfact = bm.evaluate(test_x[0:num_test_samples], np.argmax(test_y[0:num_test_samples],axis=1),model, mode='feat',aggregate=True)


No Target
No Target


In [50]:
SummaryTable_counterfact.head()

,d1_mean,d2_mean,d3_mean,d4_mean,validty_mean,d1_std,d2_std,d3_std,d4_std,validty_std,method,normalize,tsr,transformer,epochs
0,0.998382,0.652476,467.489528,2.783514,1.0,0.002288,0.063659,113.706556,0.004901,0.0,GRAD,True,True,NaN,NaN
1,0.999191,0.639725,475.515837,2.976765,1.0,0.001144,0.075973,85.324344,0.295514,0.0,FA,True,True,NaN,NaN
2,0.957929,0.329733,327.827550,3.935000,1.0,0.002288,0.141273,105.262107,1.265721,0.0,NaN,NaN,NaN,authentic_opposing_information,30


In [51]:
SummaryTable_counterfact.to_csv(f"{interp_folder}/{model_name}_CF.csv", index=False)

# Faithfulness Metrics

In [20]:
from XTSCBench.FaithfulnessEvaluation import FaithfulnessEvaluation
bm=FaithfulnessEvaluation(explainer=explainer,mlmodel=None)


In [21]:
SummaryTable_faith = bm.evaluate(test_x[0:num_test_samples], np.argmax(test_y[0:num_test_samples],axis=1), model, mode='feat',aggregate=True)


GET METRICS
Original (2, 140, 1)
EXP (2, 140, 1)
Num features=140, Mode=feat


RuntimeError: Given groups=1, weight of size [64, 1, 8], expected input[1, 140, 2] to have 1 channels, but got 140 channels instead

In [65]:
SummaryTable_faith

,method,normalize,tsr
0,GRAD,True,True


In [61]:
SummaryTable_faith.head()

,method,normalize,tsr,transformer,epochs
0,GRAD,True,True,NaN,NaN
1,FA,True,True,NaN,NaN
2,NaN,NaN,NaN,authentic_opposing_information,30


In [55]:
SummaryTable_faith.to_csv(f"{interp_folder}/{model_name}_faith.csv")

# Reliability Metrics

In [56]:
from XTSCBench.ReliabilityEvaluation import ReliabilityEvaluation
from quantus.metrics.localisation.auc import AUC

bm=ReliabilityEvaluation(explainer=explainer,mlmodel=None, metrics=[AUC()])


Warnings and information:
 (1) The AUC metric is likely to be sensitive to the choice of ground truth mask i.e., the 's_batch' input as well as if absolute values 'abs' are taken of the attributions .  
 (2) If attributions are normalised or their absolute values are taken it may destroy or skew information in the explanation and as a result, affect the overall evaluation outcome.
 (3) Make sure to validate the choices for hyperparameters of the metric (by calling .get_params of the metric instance).
 (4) For further information, see original publication: Fawcett, Tom. 'An introduction to ROC analysis' Pattern Recognition Letters Vol 27, Issue 8, (2006).
 (5) To disable these warnings set 'disable_warnings' = True when initialising the metric.



In [57]:
#CAREFUL THIS IS AN ASSUMPTION
meta=np.zeros_like(test_x[0:num_test_samples])
meta[:,10:20]= np.ones_like(meta[:,10:20])
SummaryTable_reiable = bm.evaluate(test_x[0:num_test_samples], np.argmax(test_y[0:num_test_samples],axis=1),model,meta=meta,exp=None, mode='feat',aggregate=True)


No Target
No Target


In [59]:
SummaryTable_reiable.head()

,<quantus.metrics.localisation.auc.AUC object at 0x7fbd021032d0>_mean,Pointing_mean,Relevance Rank_mean,Relevance Mass_mean,AuC_mean,<quantus.metrics.localisation.auc.AUC object at 0x7fbd021032d0>_std,Pointing_std,Relevance Rank_std,Relevance Mass_std,AuC_std,method,normalize,tsr,transformer,epochs
0,0.515505,0.0,0.050000,0.047891,0.515505,0.011344,0.0,0.02357,0.000871,0.011344,GRAD,True,True,NaN,NaN
1,0.511990,0.0,0.033333,0.046066,0.511990,0.030144,0.0,0.04714,0.010573,0.030144,FA,True,True,NaN,NaN
2,0.622194,0.0,0.066667,-0.712406,0.622194,0.000601,0.0,0.00000,0.977226,0.000601,NaN,NaN,NaN,authentic_opposing_information,30


In [60]:
SummaryTable_reiable.to_csv(f"{interp_folder}/{model_name}_reliable.csv", index=False)

# Complexity Metrics

In [76]:
from XTSCBench.ComplexityEvaluation import ComplexityEvaluation
from quantus.metrics.complexity.effective_complexity import EffectiveComplexity

bm=ComplexityEvaluation(explainer=explainer, metrics= [EffectiveComplexity()])


Warnings and information:
 (1) The Effective Complexity metric is likely to be sensitive to the choice of normalising 'normalise' (and 'normalise_func') and if taking absolute values of attributions 'abs' and the choice of threshold 'eps'.  
 (2) If attributions are normalised or their absolute values are taken it may destroy or skew information in the explanation and as a result, affect the overall evaluation outcome.
 (3) Make sure to validate the choices for hyperparameters of the metric (by calling .get_params of the metric instance).
 (4) For further information, see original publication: Nguyen, An-phi, and María Rodríguez Martínez. 'On quantitative aspects of model interpretability.' arXiv preprint arXiv:2007.07584 (2020)..
 (5) To disable these warnings set 'disable_warnings' = True when initialising the metric.



In [77]:
SummaryTable_complex = bm.evaluate(test_x[0:num_test_samples], np.argmax(test_y[0:num_test_samples],axis=1), model, mode='feat',aggregate=True)


No Target
No Target


In [79]:
SummaryTable_complex

,complexity_mean,<quantus.metrics.complexity.effective_complexity.EffectiveComplexity object at 0x7f89a761b350>_mean,complexity_std,<quantus.metrics.complexity.effective_complexity.EffectiveComplexity object at 0x7f89a761b350>_std,method,normalize,tsr,transformer,epochs
0,5.361227,522.0,0.161484,0.000000,GRAD,True,True,NaN,NaN
1,5.330293,522.0,0.171294,0.000000,FA,True,True,NaN,NaN
2,6.100786,617.0,0.045401,1.414214,NaN,NaN,NaN,authentic_opposing_information,30


In [80]:
SummaryTable_complex.to_csv(f"{interp_folder}/{model_name}_complexity.csv", index=False)

# Robustness Metrics

In [45]:
from XTSCBench.RobustnessEvaluation import RobustnessEvaluation

bm=RobustnessEvaluation(explainer=explainer,mlmodel=None)


In [46]:
SummaryTable_robust = bm.evaluate(test_x[0:2], np.argmax(test_y[0:2],axis=1), model,exp=None, mode='time',aggregate=True)

No Target
No Target
Robustness Shapes
(2, 96, 1)
[1, 1]
X1  (1, 96)
y1  1
No Target


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

In [ ]:
SummaryTable_robust.head()